In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler

from imblearn.over_sampling import SMOTE

import tensorflow as tf
from tensorflow import keras

import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('telecom_churn_dataset.csv')

In [2]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [3]:
df.drop(['customerID'],axis=1,inplace=True)

In [4]:
df.columns

Index(['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure',
       'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
       'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
       'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod',
       'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

In [5]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'],errors='coerce')

In [6]:
df.dropna(inplace=True)

In [7]:
df.columns

Index(['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure',
       'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
       'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
       'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod',
       'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

In [8]:
df.dtypes

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges        float64
Churn                object
dtype: object

In [9]:
for i in df:
    print(f'{i} : {df[i].unique()}')

gender : ['Female' 'Male']
SeniorCitizen : [0 1]
Partner : ['Yes' 'No']
Dependents : ['No' 'Yes']
tenure : [ 1 34  2 45  8 22 10 28 62 13 16 58 49 25 69 52 71 21 12 30 47 72 17 27
  5 46 11 70 63 43 15 60 18 66  9  3 31 50 64 56  7 42 35 48 29 65 38 68
 32 55 37 36 41  6  4 33 67 23 57 61 14 20 53 40 59 24 44 19 54 51 26 39]
PhoneService : ['No' 'Yes']
MultipleLines : ['No phone service' 'No' 'Yes']
InternetService : ['DSL' 'Fiber optic' 'No']
OnlineSecurity : ['No' 'Yes' 'No internet service']
OnlineBackup : ['Yes' 'No' 'No internet service']
DeviceProtection : ['No' 'Yes' 'No internet service']
TechSupport : ['No' 'Yes' 'No internet service']
StreamingTV : ['No' 'Yes' 'No internet service']
StreamingMovies : ['No' 'Yes' 'No internet service']
Contract : ['Month-to-month' 'One year' 'Two year']
PaperlessBilling : ['Yes' 'No']
PaymentMethod : ['Electronic check' 'Mailed check' 'Bank transfer (automatic)'
 'Credit card (automatic)']
MonthlyCharges : [29.85 56.95 53.85 ... 63.1  44.2  78

In [10]:
df.replace('No internet service','No',inplace=True)
df.replace('No phone service','No',inplace=True)

In [11]:
for i in df:
    print(f'{i} : {df[i].unique()}')

gender : ['Female' 'Male']
SeniorCitizen : [0 1]
Partner : ['Yes' 'No']
Dependents : ['No' 'Yes']
tenure : [ 1 34  2 45  8 22 10 28 62 13 16 58 49 25 69 52 71 21 12 30 47 72 17 27
  5 46 11 70 63 43 15 60 18 66  9  3 31 50 64 56  7 42 35 48 29 65 38 68
 32 55 37 36 41  6  4 33 67 23 57 61 14 20 53 40 59 24 44 19 54 51 26 39]
PhoneService : ['No' 'Yes']
MultipleLines : ['No' 'Yes']
InternetService : ['DSL' 'Fiber optic' 'No']
OnlineSecurity : ['No' 'Yes']
OnlineBackup : ['Yes' 'No']
DeviceProtection : ['No' 'Yes']
TechSupport : ['No' 'Yes']
StreamingTV : ['No' 'Yes']
StreamingMovies : ['No' 'Yes']
Contract : ['Month-to-month' 'One year' 'Two year']
PaperlessBilling : ['Yes' 'No']
PaymentMethod : ['Electronic check' 'Mailed check' 'Bank transfer (automatic)'
 'Credit card (automatic)']
MonthlyCharges : [29.85 56.95 53.85 ... 63.1  44.2  78.7 ]
TotalCharges : [  29.85 1889.5   108.15 ...  346.45  306.6  6844.5 ]
Churn : ['No' 'Yes']


In [12]:
df['gender'].replace({'Female' : 0, 'Male' : 1},inplace=True)

In [13]:
yes_or_no = ['Partner','Dependents','PhoneService','MultipleLines','OnlineSecurity','OnlineBackup',
  'DeviceProtection','TechSupport','StreamingTV','StreamingMovies','PaperlessBilling',
  'Churn']
for i in yes_or_no:
    df[i].replace({'Yes':1, 'No':0},inplace=True)

In [14]:
for i in df:
    print(f'{i} : {df[i].unique()}')

gender : [0 1]
SeniorCitizen : [0 1]
Partner : [1 0]
Dependents : [0 1]
tenure : [ 1 34  2 45  8 22 10 28 62 13 16 58 49 25 69 52 71 21 12 30 47 72 17 27
  5 46 11 70 63 43 15 60 18 66  9  3 31 50 64 56  7 42 35 48 29 65 38 68
 32 55 37 36 41  6  4 33 67 23 57 61 14 20 53 40 59 24 44 19 54 51 26 39]
PhoneService : [0 1]
MultipleLines : [0 1]
InternetService : ['DSL' 'Fiber optic' 'No']
OnlineSecurity : [0 1]
OnlineBackup : [1 0]
DeviceProtection : [0 1]
TechSupport : [0 1]
StreamingTV : [0 1]
StreamingMovies : [0 1]
Contract : ['Month-to-month' 'One year' 'Two year']
PaperlessBilling : [1 0]
PaymentMethod : ['Electronic check' 'Mailed check' 'Bank transfer (automatic)'
 'Credit card (automatic)']
MonthlyCharges : [29.85 56.95 53.85 ... 63.1  44.2  78.7 ]
TotalCharges : [  29.85 1889.5   108.15 ...  346.45  306.6  6844.5 ]
Churn : [0 1]


In [15]:
df['InternetService'].replace({'DSL':0, 'Fiber optic':1, 'No':2},inplace=True)
df['Contract'].replace({'Month-to-month':0, 'One year':1, 'Two year':2},inplace=True)
df['PaymentMethod'].replace({"Electronic check":0, "Mailed check":1, "Bank transfer (automatic)":2,"Credit card (automatic)":3},inplace=True)

In [16]:
for i in df:
    print(f'{i} : {df[i].unique()}')

gender : [0 1]
SeniorCitizen : [0 1]
Partner : [1 0]
Dependents : [0 1]
tenure : [ 1 34  2 45  8 22 10 28 62 13 16 58 49 25 69 52 71 21 12 30 47 72 17 27
  5 46 11 70 63 43 15 60 18 66  9  3 31 50 64 56  7 42 35 48 29 65 38 68
 32 55 37 36 41  6  4 33 67 23 57 61 14 20 53 40 59 24 44 19 54 51 26 39]
PhoneService : [0 1]
MultipleLines : [0 1]
InternetService : [0 1 2]
OnlineSecurity : [0 1]
OnlineBackup : [1 0]
DeviceProtection : [0 1]
TechSupport : [0 1]
StreamingTV : [0 1]
StreamingMovies : [0 1]
Contract : [0 1 2]
PaperlessBilling : [1 0]
PaymentMethod : [0 1 2 3]
MonthlyCharges : [29.85 56.95 53.85 ... 63.1  44.2  78.7 ]
TotalCharges : [  29.85 1889.5   108.15 ...  346.45  306.6  6844.5 ]
Churn : [0 1]


In [17]:
df.dtypes

gender                int64
SeniorCitizen         int64
Partner               int64
Dependents            int64
tenure                int64
PhoneService          int64
MultipleLines         int64
InternetService       int64
OnlineSecurity        int64
OnlineBackup          int64
DeviceProtection      int64
TechSupport           int64
StreamingTV           int64
StreamingMovies       int64
Contract              int64
PaperlessBilling      int64
PaymentMethod         int64
MonthlyCharges      float64
TotalCharges        float64
Churn                 int64
dtype: object

In [18]:
z=df.drop(['Churn'],axis=1)
y=df['Churn']

In [19]:
z.shape

(7032, 19)

In [20]:
ztrain,ztest,ytrain,ytest=train_test_split(z,y)

In [21]:
ann = keras.Sequential([
    keras.layers.Dense(19, input_shape=(19,),activation='relu'),
    keras.layers.Dense(10,activation='relu'),
    keras.layers.Dense(1,activation='sigmoid')
])

ann.compile(optimizer='adam',
           loss='binary_crossentropy',
           metrics=['accuracy'])

ann.fit(ztrain,ytrain,epochs=100)

Epoch 1/100
165/165 [==============================] - 1s 1ms/step - loss: 26.9544 - accuracy: 0.7081
Epoch 2/100
165/165 [==============================] - 0s 1ms/step - loss: 0.7440 - accuracy: 0.7367
Epoch 3/100
165/165 [==============================] - 0s 1ms/step - loss: 0.8259 - accuracy: 0.7385
Epoch 4/100
165/165 [==============================] - 0s 1ms/step - loss: 0.6331 - accuracy: 0.7652
Epoch 5/100
165/165 [==============================] - 0s 1ms/step - loss: 0.7111 - accuracy: 0.7582
Epoch 6/100
165/165 [==============================] - 0s 1ms/step - loss: 0.5698 - accuracy: 0.7744
Epoch 7/100
165/165 [==============================] - 0s 1ms/step - loss: 0.5156 - accuracy: 0.7823
Epoch 8/100
165/165 [==============================] - 0s 1ms/step - loss: 0.5247 - accuracy: 0.7799
Epoch 9/100
165/165 [==============================] - 0s 1ms/step - loss: 0.5725 - accuracy: 0.7722
Epoch 10/100
165/165 [==============================] - 0s 949us/step - loss: 0.8342 - acc

165/165 [==============================] - 0s 1ms/step - loss: 0.5180 - accuracy: 0.7852
Epoch 82/100
165/165 [==============================] - 0s 1ms/step - loss: 0.5776 - accuracy: 0.7663
Epoch 83/100
165/165 [==============================] - 0s 987us/step - loss: 0.5692 - accuracy: 0.7754
Epoch 84/100
165/165 [==============================] - 0s 1ms/step - loss: 0.5372 - accuracy: 0.7817
Epoch 85/100
165/165 [==============================] - 0s 1ms/step - loss: 0.6426 - accuracy: 0.7651
Epoch 86/100
165/165 [==============================] - 0s 1ms/step - loss: 0.6769 - accuracy: 0.7762
Epoch 87/100
165/165 [==============================] - 0s 1ms/step - loss: 0.6006 - accuracy: 0.7845
Epoch 88/100
165/165 [==============================] - 0s 1ms/step - loss: 0.5086 - accuracy: 0.7842
Epoch 89/100
165/165 [==============================] - 0s 1ms/step - loss: 0.6183 - accuracy: 0.7782
Epoch 90/100
165/165 [==============================] - 0s 990us/step - loss: 0.6630 - accura

In [22]:
ann.evaluate(ztest,ytest)

y_pred = ann.predict(ztest)
y_pred = np.round(y_pred)
print("Classification Report: \n", classification_report(ytest, y_pred))

55/55 [==============================] - 0s 1ms/step - loss: 0.6341 - accuracy: 0.7776
Classification Report: 
               precision    recall  f1-score   support

           0       0.79      0.95      0.86      1281
           1       0.71      0.30      0.43       477

    accuracy                           0.78      1758
   macro avg       0.75      0.63      0.64      1758
weighted avg       0.77      0.78      0.74      1758



#### Evaluating with MinMaScaler

In [23]:
df2 = df
mms_cols = ['tenure','MonthlyCharges','TotalCharges']
mms=MinMaxScaler()
df2[mms_cols] = mms.fit_transform(df2[mms_cols])

In [24]:
z1=df2.drop(['Churn'],axis=1)
y1=df2['Churn']
ztrain1,ztest1,ytrain1,ytest1=train_test_split(z1,y1)

In [25]:
ann = keras.Sequential([
    keras.layers.Dense(19, input_shape=(19,),activation='relu'),
    keras.layers.Dense(10,activation='relu'),
    keras.layers.Dense(1,activation='sigmoid')
])

ann.compile(optimizer='adam',
           loss='binary_crossentropy',
           metrics=['accuracy'])

ann.fit(ztrain1,ytrain1,epochs=100)

Epoch 1/100
165/165 [==============================] - 1s 1ms/step - loss: 0.6448 - accuracy: 0.5588
Epoch 2/100
165/165 [==============================] - 0s 1ms/step - loss: 0.4492 - accuracy: 0.7644
Epoch 3/100
165/165 [==============================] - 0s 1ms/step - loss: 0.4317 - accuracy: 0.7827
Epoch 4/100
165/165 [==============================] - 0s 1ms/step - loss: 0.4304 - accuracy: 0.7898
Epoch 5/100
165/165 [==============================] - 0s 977us/step - loss: 0.4265 - accuracy: 0.7994
Epoch 6/100
165/165 [==============================] - 0s 943us/step - loss: 0.4270 - accuracy: 0.7933
Epoch 7/100
165/165 [==============================] - 0s 1ms/step - loss: 0.4077 - accuracy: 0.8078
Epoch 8/100
165/165 [==============================] - 0s 989us/step - loss: 0.4015 - accuracy: 0.8122
Epoch 9/100
165/165 [==============================] - 0s 1ms/step - loss: 0.4080 - accuracy: 0.8095
Epoch 10/100
165/165 [==============================] - 0s 1ms/step - loss: 0.4124 - 

165/165 [==============================] - 0s 1ms/step - loss: 0.3634 - accuracy: 0.8384
Epoch 82/100
165/165 [==============================] - 0s 1ms/step - loss: 0.3740 - accuracy: 0.8308
Epoch 83/100
165/165 [==============================] - 0s 1ms/step - loss: 0.3744 - accuracy: 0.8259
Epoch 84/100
165/165 [==============================] - 0s 972us/step - loss: 0.3676 - accuracy: 0.8352
Epoch 85/100
165/165 [==============================] - 0s 1ms/step - loss: 0.3791 - accuracy: 0.8287
Epoch 86/100
165/165 [==============================] - 0s 1ms/step - loss: 0.3745 - accuracy: 0.8223
Epoch 87/100
165/165 [==============================] - 0s 948us/step - loss: 0.3743 - accuracy: 0.8252
Epoch 88/100
165/165 [==============================] - 0s 1ms/step - loss: 0.3710 - accuracy: 0.8371
Epoch 89/100
165/165 [==============================] - 0s 1ms/step - loss: 0.3690 - accuracy: 0.8300
Epoch 90/100
165/165 [==============================] - 0s 958us/step - loss: 0.3824 - accu

In [26]:
ann.evaluate(ztest,ytest)

y_pred = ann.predict(ztest)
y_pred = np.round(y_pred)
print("Classification Report: \n", classification_report(ytest, y_pred))

55/55 [==============================] - 0s 1ms/step - loss: 113.0015 - accuracy: 0.2725
Classification Report: 
               precision    recall  f1-score   support

           0       0.51      0.06      0.10      1281
           1       0.25      0.85      0.39       477

    accuracy                           0.27      1758
   macro avg       0.38      0.45      0.25      1758
weighted avg       0.44      0.27      0.18      1758



### Imbalanced dataset techniques

#### Functions for the program

In [27]:
def train_test(df,n):
    
    print('Churn value count : \n')
    
    print(df['Churn'].value_counts())
    
    z=df2.drop(['Churn'],axis=1)   
    y=df2['Churn']
    
    from sklearn.model_selection import train_test_split
    ztrain,ztest,ytrain,ytest = train_test_split(z,y)
    
    print("\nShape of train test split\n")
    
    print(ztrain.shape)
    print(ztest.shape)
    print(ytrain.shape)
    print(ytest.shape)

    print('\n Neural network starts \n')
    
    ann_regular = keras.Sequential([
        keras.layers.Dense(19, input_shape=(19,),activation='relu'),
        keras.layers.Dense(10,activation='relu'),
        keras.layers.Dense(1,activation='sigmoid')
    ])
    
    ann_regular.compile(optimizer='adam',
                   loss='binary_crossentropy',
                   metrics=['accuracy'])
    
    ann_regular.fit(ztrain,ytrain,epochs=n)
    
    y_pred = ann_regular.predict(ztest)
    y_pred = np.round(y_pred)
    
    return print("Classification Report: \n", classification_report(ytest, y_pred))

#### Imbalanced dataset : Undersampling technique

In [28]:
df2_0_count,df2_1_count = df2['Churn'].value_counts()
df2_0_count,df2_1_count

(5163, 1869)

In [29]:
df2_0,df2_1 = df2[df2['Churn']==0], df2[df2['Churn']==1]

In [30]:
df2_0_class = df2_0.sample(df2_1_count)
df_under = pd.concat([df2_0_class,df2_1],axis=0)
print(df_under['Churn'].value_counts())
df_under.shape

1    1869
0    1869
Name: Churn, dtype: int64


(3738, 20)

In [31]:
train_test(df_under,100)

Churn value count : 

1    1869
0    1869
Name: Churn, dtype: int64

Shape of train test split

(5274, 19)
(1758, 19)
(5274,)
(1758,)

 Neural network starts 

Epoch 1/100
165/165 [==============================] - 1s 1ms/step - loss: 0.5751 - accuracy: 0.7068
Epoch 2/100
165/165 [==============================] - 0s 1ms/step - loss: 0.4483 - accuracy: 0.7769
Epoch 3/100
165/165 [==============================] - 0s 1ms/step - loss: 0.4326 - accuracy: 0.7887
Epoch 4/100
165/165 [==============================] - 0s 1ms/step - loss: 0.4262 - accuracy: 0.7893
Epoch 5/100
165/165 [==============================] - 0s 1ms/step - loss: 0.4298 - accuracy: 0.7858
Epoch 6/100
165/165 [==============================] - 0s 1ms/step - loss: 0.4199 - accuracy: 0.7966
Epoch 7/100
165/165 [==============================] - 0s 1ms/step - loss: 0.4105 - accuracy: 0.8071
Epoch 8/100
165/165 [==============================] - 0s 1ms/step - loss: 0.4239 - accuracy: 0.7989
Epoch 9/100
165/165 [===========

165/165 [==============================] - 0s 1ms/step - loss: 0.3786 - accuracy: 0.8198
Epoch 80/100
165/165 [==============================] - 0s 1ms/step - loss: 0.3911 - accuracy: 0.8147
Epoch 81/100
165/165 [==============================] - 0s 1ms/step - loss: 0.3951 - accuracy: 0.8169
Epoch 82/100
165/165 [==============================] - 0s 1ms/step - loss: 0.3957 - accuracy: 0.8096
Epoch 83/100
165/165 [==============================] - 0s 1ms/step - loss: 0.3817 - accuracy: 0.8173
Epoch 84/100
165/165 [==============================] - 0s 1ms/step - loss: 0.3826 - accuracy: 0.8245
Epoch 85/100
165/165 [==============================] - 0s 1ms/step - loss: 0.3990 - accuracy: 0.8146
Epoch 86/100
165/165 [==============================] - 0s 1ms/step - loss: 0.3834 - accuracy: 0.8196
Epoch 87/100
165/165 [==============================] - 0s 1ms/step - loss: 0.3927 - accuracy: 0.8226
Epoch 88/100
165/165 [==============================] - 0s 1ms/step - loss: 0.3957 - accuracy: 

### Imbalanced dataset : Oversampling technique

In [32]:
df_0_count,df_1_count = df['Churn'].value_counts()
df_0_count,df_1_count

(5163, 1869)

In [33]:
df_0,df_1 = df[df['Churn']==0],df[df['Churn']==1]

In [34]:
df_1_class_over = df_1.sample(df_0_count,replace=True)
df_over = pd.concat([df_0,df_1_class_over],axis=0)
df_over['Churn'].value_counts()

1    5163
0    5163
Name: Churn, dtype: int64

In [35]:
train_test(df_over,100)

Churn value count : 

1    5163
0    5163
Name: Churn, dtype: int64

Shape of train test split

(5274, 19)
(1758, 19)
(5274,)
(1758,)

 Neural network starts 

Epoch 1/100
165/165 [==============================] - 1s 1ms/step - loss: 0.5454 - accuracy: 0.7322
Epoch 2/100
165/165 [==============================] - 0s 1ms/step - loss: 0.4663 - accuracy: 0.7570
Epoch 3/100
165/165 [==============================] - 0s 1ms/step - loss: 0.4477 - accuracy: 0.7713
Epoch 4/100
165/165 [==============================] - 0s 1ms/step - loss: 0.4467 - accuracy: 0.7771
Epoch 5/100
165/165 [==============================] - 0s 1ms/step - loss: 0.4345 - accuracy: 0.7859
Epoch 6/100
165/165 [==============================] - 0s 1ms/step - loss: 0.4337 - accuracy: 0.7900
Epoch 7/100
165/165 [==============================] - 0s 1ms/step - loss: 0.4297 - accuracy: 0.7880
Epoch 8/100
165/165 [==============================] - 0s 1ms/step - loss: 0.4294 - accuracy: 0.7996
Epoch 9/100
165/165 [===========

165/165 [==============================] - 0s 1ms/step - loss: 0.3742 - accuracy: 0.8266
Epoch 80/100
165/165 [==============================] - 0s 1ms/step - loss: 0.3892 - accuracy: 0.8167
Epoch 81/100
165/165 [==============================] - 0s 1ms/step - loss: 0.3918 - accuracy: 0.8212
Epoch 82/100
165/165 [==============================] - 0s 1ms/step - loss: 0.3927 - accuracy: 0.8164
Epoch 83/100
165/165 [==============================] - 0s 1ms/step - loss: 0.3779 - accuracy: 0.8216
Epoch 84/100
165/165 [==============================] - 0s 1ms/step - loss: 0.3941 - accuracy: 0.8159
Epoch 85/100
165/165 [==============================] - 0s 1ms/step - loss: 0.3870 - accuracy: 0.8187
Epoch 86/100
165/165 [==============================] - 0s 1ms/step - loss: 0.3757 - accuracy: 0.8208
Epoch 87/100
165/165 [==============================] - 0s 2ms/step - loss: 0.3798 - accuracy: 0.8192
Epoch 88/100
165/165 [==============================] - 0s 2ms/step - loss: 0.3951 - accuracy: 

#### Imbalanced Dataset Method : SMOTE

In [36]:
smote = SMOTE(sampling_strategy='minority')
z_sm, y_sm = smote.fit_sample(z1,y1)

y_sm.value_counts()

1    5163
0    5163
Name: Churn, dtype: int64

In [37]:
from sklearn.model_selection import train_test_split
ztrain1,ztest1,ytrain1,ytest1 = train_test_split(z_sm, y_sm)
print("\nShape of train test split\n")

print(ztrain1.shape)
print(ztest1.shape)
print(ytrain1.shape)
print(ytest1.shape)

print('\n Neural network starts\n')

ann_regular = keras.Sequential([
    keras.layers.Dense(19, input_shape=(19,),activation='relu'),
    keras.layers.Dense(10,activation='relu'),
    keras.layers.Dense(1,activation='sigmoid')
])
ann_regular.compile(optimizer='adam',
loss='binary_crossentropy',
metrics=['accuracy'])

ann_regular.fit(ztrain1,ytrain1,epochs=100)


Shape of train test split

(7744, 19)
(2582, 19)
(7744,)
(2582,)

 Neural network starts

Epoch 1/100
242/242 [==============================] - 1s 1ms/step - loss: 0.6564 - accuracy: 0.6201
Epoch 2/100
242/242 [==============================] - 0s 1ms/step - loss: 0.5045 - accuracy: 0.7510
Epoch 3/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4845 - accuracy: 0.7655
Epoch 4/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4834 - accuracy: 0.7709
Epoch 5/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4630 - accuracy: 0.7805
Epoch 6/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4547 - accuracy: 0.7836
Epoch 7/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4544 - accuracy: 0.7955
Epoch 8/100
242/242 [==============================] - 0s 2ms/step - loss: 0.4523 - accuracy: 0.7907
Epoch 9/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4443 - accuracy: 0.7989


242/242 [==============================] - 0s 1ms/step - loss: 0.4010 - accuracy: 0.8219
Epoch 81/100
242/242 [==============================] - 0s 1ms/step - loss: 0.3967 - accuracy: 0.8276
Epoch 82/100
242/242 [==============================] - 0s 1ms/step - loss: 0.3934 - accuracy: 0.8289
Epoch 83/100
242/242 [==============================] - 0s 1ms/step - loss: 0.4009 - accuracy: 0.8238
Epoch 84/100
242/242 [==============================] - 0s 1ms/step - loss: 0.3949 - accuracy: 0.8280
Epoch 85/100
242/242 [==============================] - 0s 1ms/step - loss: 0.3858 - accuracy: 0.8299
Epoch 86/100
242/242 [==============================] - 0s 1ms/step - loss: 0.3968 - accuracy: 0.8210
Epoch 87/100
242/242 [==============================] - 0s 1ms/step - loss: 0.3966 - accuracy: 0.8232
Epoch 88/100
242/242 [==============================] - 0s 1ms/step - loss: 0.3960 - accuracy: 0.8318
Epoch 89/100
242/242 [==============================] - 0s 1ms/step - loss: 0.3911 - accuracy: 

In [38]:
ann_regular.evaluate(ztest1,ytest1)

81/81 [==============================] - 0s 1ms/step - loss: 0.4294 - accuracy: 0.8056


[0.4293861389160156, 0.8055770993232727]

In [39]:
y_pred1 = ann_regular.predict(ztest1)
y_pred1 = np.round(y_pred1)

print("Classification Report: \n", classification_report(ytest1, y_pred1))

Classification Report: 
               precision    recall  f1-score   support

           0       0.83      0.76      0.79      1265
           1       0.79      0.85      0.82      1317

    accuracy                           0.81      2582
   macro avg       0.81      0.80      0.80      2582
weighted avg       0.81      0.81      0.81      2582

